# Customers Data Preparation
---
---

#### Importing the libraries

In [206]:
import pandas as pd
import numpy as np
import seaborn as sns

import sklearn
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import os.path


#### Import data

In [207]:
customers = pd.read_csv('/Users/patrick/Desktop/Lighthouse_labs/Data Projects/Clustering-/Datasets/twm_customer.csv', delimiter=';')

In [208]:
customers.head(5)

,cust_id,income,age,years_with_bank,nbr_children,gender,marital_status,name_prefix,first_name,last_name,street_nbr,street_name,postal_code,city_name,state_code
0,1362691,26150,46,5,1,M,2,,Donald ...,Marek ...,8298,Second ...,89194,Las Vegas,NV
1,1362487,6605,71,1,0,M,2,,ChingDyi ...,Moussavi ...,10603,Daffodil ...,90159,Los Angeles,CA
2,1363160,18548,38,8,0,F,1,,Rosa ...,Johnston ...,8817,Figueroa ...,90024,Los Angeles,CA
3,1362752,47668,54,3,0,F,1,,Lisa ...,Martin ...,676,Humble ...,90172,Los Angeles,CA
4,1362548,44554,59,9,2,F,4,,Barbara ...,O'Malley ...,6578,C ...,10138,New York City,NY


In [209]:
# data sctructure
customers.describe()

,cust_id,income,age,years_with_bank,nbr_children,marital_status,street_nbr,postal_code
count,7.470000e+02,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000
mean,1.362991e+06,22728.281124,42.479250,3.907631,0.714859,1.882195,8523.109772,59502.690763
std,2.925255e+02,22207.221405,19.114879,2.675634,1.103410,0.892051,5219.488263,31205.926508
min,1.362480e+06,0.000000,13.000000,0.000000,0.000000,1.000000,196.000000,2149.000000
25%,1.362732e+06,7118.500000,28.000000,2.000000,0.000000,1.000000,4295.500000,33145.000000
50%,1.362993e+06,17242.000000,42.000000,4.000000,0.000000,2.000000,8001.000000,64132.000000
75%,1.363244e+06,31338.000000,56.000000,6.000000,1.000000,2.000000,12039.000000,90082.000000
max,1.363495e+06,144157.000000,89.000000,9.000000,5.000000,4.000000,19940.000000,98186.000000


In [210]:
# check for null values
customers.isnull().sum()

cust_id            0
income             0
age                0
years_with_bank    0
nbr_children       0
gender             0
marital_status     0
name_prefix        0
first_name         0
last_name          0
street_nbr         0
street_name        0
postal_code        0
city_name          0
state_code         0
dtype: int64

In [211]:
# check for duplicates
customers.duplicated().sum()

0

In [212]:
# check for empty values
customers.isna().sum()

cust_id            0
income             0
age                0
years_with_bank    0
nbr_children       0
gender             0
marital_status     0
name_prefix        0
first_name         0
last_name          0
street_nbr         0
street_name        0
postal_code        0
city_name          0
state_code         0
dtype: int64

In [213]:
# Remove redundant columns
customers = customers.drop(['name_prefix', 'first_name', 'last_name', 'street_name', 'street_nbr', 'postal_code', 'state_code'], axis=1)

In [214]:
# describe customers again
customers.describe()

,cust_id,income,age,years_with_bank,nbr_children,marital_status
count,7.470000e+02,747.000000,747.000000,747.000000,747.000000,747.000000
mean,1.362991e+06,22728.281124,42.479250,3.907631,0.714859,1.882195
std,2.925255e+02,22207.221405,19.114879,2.675634,1.103410,0.892051
min,1.362480e+06,0.000000,13.000000,0.000000,0.000000,1.000000
25%,1.362732e+06,7118.500000,28.000000,2.000000,0.000000,1.000000
50%,1.362993e+06,17242.000000,42.000000,4.000000,0.000000,2.000000
75%,1.363244e+06,31338.000000,56.000000,6.000000,1.000000,2.000000
max,1.363495e+06,144157.000000,89.000000,9.000000,5.000000,4.000000


In [215]:
customers.head(5)

,cust_id,income,age,years_with_bank,nbr_children,gender,marital_status,city_name
0,1362691,26150,46,5,1,M,2,Las Vegas
1,1362487,6605,71,1,0,M,2,Los Angeles
2,1363160,18548,38,8,0,F,1,Los Angeles
3,1362752,47668,54,3,0,F,1,Los Angeles
4,1362548,44554,59,9,2,F,4,New York City


In [216]:
# check for duplicate customer id's using len() and print the number of duplicates
print(len(customers['cust_id']) - len(customers['cust_id'].unique()))

0


In [217]:
# missing values percent in columns
def missing_values_table(df):
        '''function that checks dataframe for missing values and returns a table with the results'''
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        print(mis_val_table_ren_columns)


In [218]:
missing_values_table (customers)

                 Missing Values  % of Total Values
cust_id                       0                0.0
income                        0                0.0
age                           0                0.0
years_with_bank               0                0.0
nbr_children                  0                0.0
gender                        0                0.0
marital_status                0                0.0
city_name                     0                0.0


In [219]:
# drop cust_id column
customers = customers.drop(['cust_id'], axis=1)

In [220]:
customers.head(5)

,income,age,years_with_bank,nbr_children,gender,marital_status,city_name
0,26150,46,5,1,M,2,Las Vegas
1,6605,71,1,0,M,2,Los Angeles
2,18548,38,8,0,F,1,Los Angeles
3,47668,54,3,0,F,1,Los Angeles
4,44554,59,9,2,F,4,New York City


#### Scaling parameters

In [233]:
# import MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
# import standard scaler
from sklearn.preprocessing import StandardScaler

In [234]:
# scale number of children column using standard scaler
scaler = StandardScaler()
customers['nbr_children'] = scaler.fit_transform(customers[['nbr_children']])

In [223]:
# get dummies for marital status
customers = pd.get_dummies(customers, columns=['marital_status'])

In [235]:
# scale age column using MinMaxScaler to 1 decimal place
customers['age'] = scaler.fit_transform(customers[['age']])

In [236]:
# scale years with bank column using MinMaxScaler to 1 decimal place
customers['years_with_bank'] = scaler.fit_transform(customers[['years_with_bank']])

In [237]:
customers.head(5)

,income,age,years_with_bank,nbr_children,marital_status_1,marital_status_2,marital_status_3,marital_status_4
0,26150,0.184312,0.408539,0.258591,0,1,0,0
1,6605,1.493070,-1.087435,-0.648298,0,1,0,0
2,18548,-0.234490,1.530520,-0.648298,1,0,0,0
3,47668,0.603115,-0.339448,-0.648298,1,0,0,0
4,44554,0.864867,1.904514,1.165479,0,0,0,1


In [238]:
#scale income column using MinMaxScaler
customers['income'] = scaler.fit_transform(customers[['income']])

In [228]:
# drop cit_name column
customers = customers.drop(['city_name'], axis=1)

In [229]:
customers = customers.drop(['gender'], axis=1)

#### Feature selection

In [230]:
# using correlation matrix
customers.corr()

,income,age,years_with_bank,nbr_children,marital_status_1,marital_status_2,marital_status_3,marital_status_4
income,1.000000,0.244919,0.040604,0.283352,-0.295303,0.164599,0.174053,0.060656
age,0.244919,1.000000,0.181478,-0.057195,-0.502998,0.412111,0.054015,0.081631
years_with_bank,0.040604,0.181478,1.000000,-0.034814,-0.027494,0.055764,-0.025657,-0.028405
nbr_children,0.283352,-0.057195,-0.034814,1.000000,-0.496271,0.256925,0.151931,0.253954
marital_status_1,-0.295303,-0.502998,-0.027494,-0.496271,1.000000,-0.724575,-0.200598,-0.246149
marital_status_2,0.164599,0.412111,0.055764,0.256925,-0.724575,1.000000,-0.248040,-0.304364
marital_status_3,0.174053,0.054015,-0.025657,0.151931,-0.200598,-0.248040,1.000000,-0.084263
marital_status_4,0.060656,0.081631,-0.028405,0.253954,-0.246149,-0.304364,-0.084263,1.000000


In [239]:
customers.head(5)

,income,age,years_with_bank,nbr_children,marital_status_1,marital_status_2,marital_status_3,marital_status_4
0,0.154185,0.184312,0.408539,0.258591,0,1,0,0
1,-0.726524,1.493070,-1.087435,-0.648298,0,1,0,0
2,-0.188366,-0.234490,1.530520,-0.648298,1,0,0,0
3,1.123798,0.603115,-0.339448,-0.648298,1,0,0,0
4,0.983479,0.864867,1.904514,1.165479,0,0,0,1


In [240]:
customers.to_csv('customers_scaled.csv')